In [2]:
from edward.models import ConditionalDistribution

In [1]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import six
import tensorflow as tf

from edward.models import (
    Categorical, Dirichlet, Empirical, InverseGamma,
    MultivariateNormalDiag, Normal, ParamMixture, Beta, Bernoulli, Mixture, MixtureSameFamily, PointMass)

plt.style.use('ggplot')

/afs/inf.ed.ac.uk/user/s12/s1202144/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [45]:
tf.ones(D)*100

<tf.Tensor 'mul_13:0' shape=(3,) dtype=float32>

In [2]:
import data
import random

data_dict = {c:data.generate_data(c, 2) for c in ['red', 'green', 'blue', 'yellow']}
objs = {}
n = 2
for j, c in enumerate(['red', 'green', 'blue', 'yellow']):
    for i in range(n):
        objs["o{}".format(j*n + i)] = (c, data_dict[c][i])
        
        
def check_rule(o1, o2):
    c1, d1 = o1
    c2, d2 = o2
    
    if c1 == 'red': 
         if c2 != 'blue':
            return 'correction' 
    
    return 'no correction'

def correction_to_int(corr_string):
    return int(corr_string == 'correction')

def colour_to_int(corr_string, colour):
    return int(corr_string == colour)

def my_sample(objs, n=10):
    cs = []
    red = []
    blue = []
    
    for i in range(n):
        o1 = random.sample(objs.keys(), k=1)[0]
        o2 = random.sample(objs.keys(), k=1)[0]
        while o1 == o2:
            o2 = random.sample(objs.keys(), k=1)[0]
        c = correction_to_int(check_rule(objs[o1], objs[o2]))
        f_red = objs[o1][1]
        f_blue = objs[o2][1]
        cs.append(c)
        red.append(f_red)
        blue.append(f_blue)
    return cs, red, blue

def my_colour_sample(objs, n=10):
    cs = []
    red = []
    
    for i in range(n):
        o1 = random.sample(objs.keys(), k=1)[0]
        c = objs[o1][0]
        f_red = objs[o1][1]
        cs.append(c)
        red.append(f_red)
    return cs, red

colours, c_data = my_colour_sample(objs)
colours = list(map(lambda x: colour_to_int(x, 'blue'), colours))
colours = np.array(colours)
c_data = np.array(c_data)

colours

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0])

In [3]:
bs = Beta([1.0,1.0,1.0], [1.0,1.0,1.0]) # priors for red, blue and rule respectively
len_bs = 3

red_prior = Dirichlet(tf.ones(2))
blue_prior = Dirichlet(tf.ones(2))

#red = Categorical(probs=[bs[0], 1-bs[0]]) #Bernoulli(bs[0])
#blue = Categorical(probs=[bs[1], 1-bs[1]])

rule_prior = Beta([1.], [1.])
rule = Bernoulli(rule_prior)

D=3

red_mu = Normal(tf.zeros(D), tf.ones(D), sample_shape=2)
red_sigmasq = InverseGamma(tf.ones(D), tf.ones(D), sample_shape=2)


#red_c1 = MultivariateNormalDiag(tf.ones(D)*0.5, tf.ones(D)*100)
#red_c2 = MultivariateNormalDiag(red_mu, red_sigmasq)

f_red = ParamMixture(red_prior, {'loc': red_mu, 'scale_diag': tf.sqrt(red_sigmasq)},
                 MultivariateNormalDiag)

red = f_red.cat

blue_mu = Normal(tf.zeros(D), tf.ones(D), sample_shape=2)
blue_sigmasq = InverseGamma(tf.ones(D), tf.ones(D), sample_shape=2)


f_blue = ParamMixture(blue_prior, {'loc': blue_mu, 'scale_diag': tf.sqrt(blue_sigmasq)},
                 MultivariateNormalDiag)

blue = f_blue.cat

is_red = tf.cast(red, tf.bool)
is_blue = tf.cast(blue, tf.bool)
which_rule = tf.cast(rule, tf.bool)
# correction when c == 1 no correction when c == 0
correct_prob = 0.999
not_correct_prob = 1 - correct_prob
given_r_0 = tf.where(is_red, not_correct_prob, tf.where(is_blue, correct_prob, not_correct_prob)) # if r is 0 (red=>blue) then if something is not red then there is no correction  
given_r_1 = tf.where(is_blue, not_correct_prob, tf.where(is_red, correct_prob, not_correct_prob))

c_prob = tf.where(which_rule, given_r_0, given_r_1)


c = Bernoulli(probs=c_prob)

test = tf.where(tf.cast(red, tf.bool), 0.3, 0.5)

test_true = Bernoulli(probs=test)



#f_red = MixtureSameFamily(mixture_distribution=red,
#            components_distribution=MultivariateNormalDiag([mu,tf.ones(D)], [sigmasq, tf.ones(D)]))

#f_red = Mixture(cat=red, components=[
#    n,
#    Normal(mu, sigmasq)
#])
#tf.cast(red, dtype=tf.float32)*Normal(mu, sigmasq) + (tf.cast(1-red, dtype=tf.float32))*Normal(tf.ones(D, dtype=tf.float32)/2, tf.ones(D, dtype=tf.float32)*100)



sess = ed.get_session()
tf.global_variables_initializer().run()

sess.run(c.sample([1000]))"""
which_rule

<tf.Tensor 'Cast_2:0' shape=(1,) dtype=bool>

In [28]:
a = Categorical(probs=[0.3])
a.probs

<tf.Tensor 'Categorical_3/probs:0' shape=(1,) dtype=float32>

In [ ]:
from edward.models import Empirical

T = 500  # number of MCMC samples
qbs = Empirical(tf.get_variable(
    "qbs/params", [T, len_bs],
    initializer=tf.constant_initializer(1.0 / len_bs)))
qred = Empirical(tf.get_variable(
    "qmu/params", [T, K, D],
    initializer=tf.zeros_initializer()))
qsigmasq = Empirical(tf.get_variable(
    "qsigmasq/params", [T, K, D],
    initializer=tf.ones_initializer()))
qz = Empirical(tf.get_variable(
    "qz/params", [T, N],
    initializer=tf.zeros_initializer(),
    dtype=tf.int32))

In [5]:
T = 500
blue_prior = Beta(1.,1.)
blue = Bernoulli(blue_prior)
sess = ed.get_session()
tf.global_variables_initializer().run()

sess.run(blue.sample([10]))

array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0], dtype=int32)

In [6]:
blue

<ed.RandomVariable 'Bernoulli_2/' shape=() dtype=int32>

In [8]:
blue.log_prob(1)

<tf.Tensor 'Bernoulli/log_prob/Neg:0' shape=() dtype=float32>

In [9]:
sess = ed.get_session()
tf.global_variables_initializer().run()
sess.run(blue.log_prob(1))

-0.64607257

In [3]:
T = 500
with tf.variable_scope('test', reuse=tf.AUTO_REUSE):
    blue_prior = Dirichlet([1.,1.])
    blue = Categorical(probs=blue_prior, sample_shape=10)
    #qblue_prior = Empirical(tf.get_variable(
    #        "qblue_prior1/params", [T, 2],
    #        initializer=tf.constant_initializer(1)))
    qblue_prior = Dirichlet(tf.get_variable("qblue_prior/params", [2]))
    # PointMass(params=tf.Variable(tf.ones(2)*0.5)) # for MAP
    #proposal = Beta(1.,1.)

    #inference = ed.MetropolisHastings({blue_prior: qblue_prior}, {blue_prior: proposal},
    #        data={blue: colours})

#inference = ed.Gibbs({blue_prior: qblue_prior},
#        data={blue: colours})


inference = ed.KLqp({blue_prior:qblue_prior}, data={blue: colours})

inference.initialize()

sess = ed.get_session()
tf.global_variables_initializer().run()

#t_ph = tf.placeholder(tf.int32, [])
#running_cluster_means = tf.reduce_mean(qblue_mu.params[:t_ph], 0)

inference.run(n_iter=500, n_print=100, n_samples=1)

TypeError: Key-value pair in data does not have same shape: (10,), (10, 1)

In [4]:
blue_prior

<ed.RandomVariable 'test/Dirichlet/' shape=(2,) dtype=float32>

In [4]:
np.mean(sess.run(qblue_prior.sample(10)), axis=0)

array([0.48867518, 0.5113248 ], dtype=float32)

In [18]:

tf.multiply(tf.ones((3, 10)), tf.ones(10))
tf.multiply(tf.ones((10, 1)), tf.ones((10, 3)))

<tf.Tensor 'Mul_14:0' shape=(10, 3) dtype=float32>

In [3]:
T = 500
N = 10
D=3
#bb = Dirichlet(tf.ones(2))
blue_prior = Beta([1.],[1.])
#blue = Categorical(probs=[bb[0], bb[0]-1])

blue = Bernoulli(blue_prior, sample_shape=(10))

blue_mu = Normal(tf.zeros(D), tf.ones(D))
blue_sigmasq = InverseGamma(tf.ones(D), tf.ones(D))


blue_c1 = MultivariateNormalDiag(tf.ones(D)*0.5, tf.ones(D)*100, sample_shape=10)
blue_c2 = MultivariateNormalDiag(blue_mu, blue_sigmasq, sample_shape=10)

#f_blue = ParamMixture(bb, {'loc': blue_mu, 'scale_diag': tf.sqrt(blue_sigmasq)},
#                 MultivariateNormalDiag,
#                 sample_shape=N)
#blue = f_blue.cat


f_blue = tf.multiply(tf.cast(blue, tf.float32),  blue_c2) + tf.multiply(tf.cast((1 - blue), tf.float32), blue_c1)

with tf.variable_scope("test2", reuse=tf.AUTO_REUSE):
    qblue_prior = Beta(tf.get_variable("qblue_prior1/params", [1]), tf.get_variable("qblue_prior2/params", [1]))
    qblue_mu = Normal(tf.get_variable("qblue_mu/loc", [D]), tf.nn.softplus(tf.get_variable("qblue_mu/scale", [D])))
    qblue_sigmasq = Normal(tf.get_variable("qblue_sigm/loc", [D]), tf.nn.softplus(tf.get_variable("qblue_sigm/scale", [D])))
    #InverseGamma(tf.get_variable("qblue_sigmasq/loc", [D]), tf.get_variable("qblue_sigmasq/scale", [D]))

#Empirical(tf.get_variable(
        #"qblue_prior/params", [T, 2],
        #initializer=tf.constant_initializer(1)))
#qblue = Empirical(tf.get_variable(
#        "qblue/params", [T, N],
#        initializer=tf.zeros_initializer(), dtype=tf.int32))
#qblue_mu = Empirical(tf.get_variable("qblue_mu/params", [T, 2, D],
#        initializer=tf.zeros_initializer()))
#qblue_sigmasq = Empirical(tf.get_variable("qblue_sigmasq/params", [T, 2, D],
#        initializer=tf.ones_initializer()))

#colour_placeholder = tf.placeholder(tf.int32, [])
#data_placeholder = tf.placeholder(tf.float32, [D])

inference = ed.KLqp({blue_prior: qblue_prior, blue_mu:qblue_mu, blue_sigmasq: qblue_sigmasq},
                     data={blue: colours, f_blue: c_data})
inference.initialize()

sess = ed.get_session()
tf.global_variables_initializer().run()

#for i in range(500):
#    for c, d in zip(colours, c_data):
#        info_dict = inference.update({colour_placeholder: c, data_placeholder:d})
inference.run(n_iter=500, n_print=100, n_samples=10)
#        inference.print_progress(info_dict)

/afs/inf.ed.ac.uk/user/s12/s1202144/miniconda3/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/afs/inf.ed.ac.uk/user/s12/s1202144/miniconda3/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


500/500 [100%] ██████████████████████████████ Elapsed: 3s | Loss: 9.934


In [3]:
blue_prior = Dirichlet(tf.ones(2))


D=3

blue_mu = Normal(tf.zeros(D), tf.ones(D), sample_shape=2)
blue_sigmasq = InverseGamma(tf.ones(D), tf.ones(D), sample_shape=2)


f_blue = ParamMixture(blue_prior, {'loc': blue_mu, 'scale_diag': tf.sqrt(blue_sigmasq)},
                 MultivariateNormalDiag, sample_shape=10)

blue = f_blue.cat

In [4]:
with tf.variable_scope("test2", reuse=tf.AUTO_REUSE):
    qblue_prior = Dirichlet(tf.get_variable("qblue_prior1/params", [2]))#, tf.get_variable("qblue_prior2/params", [1]))
    qblue_mu = Normal(tf.get_variable("qblue_mu/loc", [2, D]), tf.nn.softplus(tf.get_variable("qblue_mu/scale", [2, D])))
    qblue_sigmasq = InverseGamma(tf.get_variable("qblue_sigm/loc", [2, D]), tf.get_variable("qblue_sigm/scale", [2, D]))
    #InverseGamma(tf.get_variable("qblue_sigmasq/loc", [D]), tf.get_variable("qblue_sigmasq/scale", [D]))

#Empirical(tf.get_variable(
        #"qblue_prior/params", [T, 2],
        #initializer=tf.constant_initializer(1)))
#qblue = Empirical(tf.get_variable(
#        "qblue/params", [T, N],
#        initializer=tf.zeros_initializer(), dtype=tf.int32))
#qblue_mu = Empirical(tf.get_variable("qblue_mu/params", [T, 2, D],
#        initializer=tf.zeros_initializer()))
#qblue_sigmasq = Empirical(tf.get_variable("qblue_sigmasq/params", [T, 2, D],
#        initializer=tf.ones_initializer()))

#colour_placeholder = tf.placeholder(tf.int32, [])
#data_placeholder = tf.placeholder(tf.float32, [D])

inference = ed.KLqp({blue_prior: qblue_prior, blue_mu:qblue_mu, blue_sigmasq: qblue_sigmasq},
                     data={blue: colours, f_blue: c_data})
inference.initialize()

sess = ed.get_session()
tf.global_variables_initializer().run()

#for i in range(500):
#    for c, d in zip(colours, c_data):
#        info_dict = inference.update({colour_placeholder: c, data_placeholder:d})
inference.run(n_iter=500, n_print=100, n_samples=10)
#        inference.print_progress(info_dict)

/afs/inf.ed.ac.uk/user/s12/s1202144/miniconda3/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/afs/inf.ed.ac.uk/user/s12/s1202144/miniconda3/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


500/500 [100%] ██████████████████████████████ Elapsed: 4s | Loss: nan


In [7]:
f_blue.conjugate_log_prob().eval()

array([[-1.047875  , -2.27665   , -0.4470823 ],
       [-1.4212389 , -3.0759904 , -0.75264436],
       [-1.1851672 , -5.6361094 , -0.8396575 ],
       [-1.0433657 , -2.0537314 , -0.75816596],
       [-1.1523949 , -1.8073025 , -3.1864033 ],
       [-1.4949081 , -1.6110331 , -0.55641264],
       [-1.1690198 , -2.032587  , -0.5858282 ],
       [-2.3730345 , -3.535501  , -0.4414657 ],
       [-1.2988878 , -3.5470862 , -1.2570676 ],
       [-1.3355322 , -2.4439044 , -0.5110403 ]], dtype=float32)

In [23]:
f_blue_post = ed.copy(f_blue, {blue_prior:qblue_prior, blue_mu: qblue_mu, blue_sigmasq:qblue_sigmasq}, copy_q=True)

In [30]:
blue_post.eval(feed_dict={f_blue:test_data})

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

In [24]:
ed.get_ancestors(f_blue_post)

[<ed.RandomVariable 'Normal_1/' shape=(2, 3) dtype=float32>,
 <ed.RandomVariable 'copied/ParamMixture_1/Categorical/' shape=(10,) dtype=int32>,
 <ed.RandomVariable 'copied/test2_2/Dirichlet/' shape=(2,) dtype=float32>,
 <ed.RandomVariable 'InverseGamma_1/' shape=(2, 3) dtype=float32>]

In [58]:
blue_post = f_blue_post.cat
blue_post
q_blue = Categorical(tf.get_variable('qblue/param3', [2]), sample_shape=10)
q_blue

<ed.RandomVariable 'Categorical_2/' shape=(10,) dtype=int32>

In [51]:
f_blue_post = ed.copy(f_blue, {blue_prior:qblue_prior, blue_mu: qblue_mu, blue_sigmasq:qblue_sigmasq}, copy_q=True)
blue_post.eval(feed_dict={f_blue:test_data})

q_blue = Categorical(tf.get_variable('qblue/param', [2]), sample_shape=10)

inference = ed.KLqp({blue_post: q_blue}, data={f_blue_post:test_data})

inference.run(n_iter=500, n_print=100)

TypeError: Key-value pair in latent_vars does not have same shape: (10,), ()

In [50]:
f_blue_post.log_prob(test_data).eval()

array([-33.556965,   0.      ], dtype=float32)

In [8]:
qblue_prior.sample(10).eval()

array([[0.1079564 ],
       [0.67778647],
       [0.2070218 ],
       [0.42886624],
       [0.2748925 ],
       [0.61006314],
       [0.14546908],
       [0.08681098],
       [0.5994823 ],
       [0.9915165 ]], dtype=float32)

In [11]:
test_c, test_data = my_colour_sample(objs)
test_c = list(map(lambda x: colour_to_int(x, 'blue'), test_c))
test_c = np.array(test_c)
test_data = np.array(test_data, dtype=np.float32)

test_c

array([0, 0, 0, 0, 1, 0, 1, 1, 1, 0])

In [4]:
sess.run(qblue_mu.sample(100))

array([[[ 5.95200479e-01,  6.06709599e-01, -7.38514811e-02],
        [-1.26881748e-02, -6.40598536e-01,  1.06572652e+00]],

       [[ 9.30757642e-01, -8.82369578e-02,  1.27150327e-01],
        [ 5.04438162e-01, -3.68533343e-01,  1.03993988e+00]],

       [[ 8.34950328e-01,  5.21271169e-01,  1.51210241e-02],
        [-5.27198136e-01,  5.02990365e-01, -4.63409424e-01]],

       [[ 1.10724592e+00,  4.02048171e-01, -1.14839986e-01],
        [-6.99628890e-01,  4.88392591e-01,  1.07865429e+00]],

       [[ 8.46085191e-01,  4.86901492e-01, -8.03241283e-02],
        [ 1.23522177e-01,  5.10799348e-01,  1.85226411e-01]],

       [[ 3.83101732e-01,  3.77910674e-01,  2.30968058e-01],
        [-3.19465399e-01,  2.09752023e-02,  7.40530014e-01]],

       [[ 9.96079743e-01,  6.40604377e-01,  3.19072753e-01],
        [ 9.16531384e-01, -9.58273411e-02,  2.14708567e+00]],

       [[ 6.62474394e-01,  6.78522646e-01, -7.70042241e-02],
        [-6.54877186e-01,  3.40716466e-02,  4.05116767e-01]],

       [

In [36]:
qblue_mu

<ed.RandomVariable 'test2_5/Normal/' shape=(3,) dtype=float32>

In [44]:
x_post = Normal(loc=qblue_mu, scale=qblue_sigmasq)
b_post = Bernoulli(qblue_prior)

In [43]:
np.mean(x_post.sample(100).eval(), axis=0)

array([ 1.4704071 ,  0.06402967, -1.7232388 ], dtype=float32)

In [34]:
K =2

# Calculate likelihood for each data point and cluster assignment,
# averaged over many posterior samples. ``x_post`` has shape (N, 100, K, D).
mu_sample = qblue_mu.sample(100)
sigmasq_sample = qblue_sigmasq.sample(100)
x_post = Normal(loc=tf.ones([N, 1, 1, 1]) * mu_sample,
                scale=tf.ones([N, 1, 1, 1]) * tf.sqrt(sigmasq_sample))
x_broadcasted = tf.tile(tf.reshape(test_data, [N, 1, 1, D]), [1, 100, K, 1])

# Sum over latent dimension, then average over posterior samples.
# ``log_liks`` ends up with shape (N, K).
log_liks = x_post.log_prob(x_broadcasted)
print(log_liks)
log_liks = tf.reduce_sum(log_liks, 3)
print(log_liks)
log_liks = tf.reduce_mean(log_liks, 1)
print(log_liks)

ValueError: Dimensions must be equal, but are 2 and 100 for 'Normal_24/log_prob/standardize/sub' (op: 'Sub') with input shapes: [10,100,2,3], [10,1,100,3].

In [15]:
clusters = tf.argmax(log_liks, 1).eval()

In [16]:
clusters

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 0])

In [89]:
objs

{'o0': ('red', (0.914986812838154, 0.04768381869734166, 0.13940673560014546)),
 'o1': ('red', (0.9183117399685538, 0.04577914562189836, 0.08039923349084321)),
 'o2': ('green',
  (0.017157724938090774, 0.9860630727959023, 0.10359268884083693)),
 'o3': ('green',
  (0.030830183424031394, 0.7999230658755758, 0.006652133607921081)),
 'o4': ('blue',
  (0.005144165464646138, 0.018402393090976205, 0.9553083359819575)),
 'o5': ('blue',
  (0.13499377025468182, 0.08355208226296003, 0.9178510538905872)),
 'o6': ('yellow',
  (0.8904197187327919, 0.9907481920038579, 0.14510569230717094)),
 'o7': ('yellow',
  (0.9151102902265296, 0.9544271428295218, 0.026391186438515537))}

In [97]:
check_rule(objs['o7'], objs['o4'])

'no correction'

In [3]:
c_data, red_data, blue_data = my_sample(objs)
c_data

NameError: name 'my_sample' is not defined

[1, 1, 0, 0, 0, 1, 0, 1, 0, 0]

In [127]:
colours

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]